In [ ]:
pip install requests beautifulsoup4

In [12]:
import requests
from bs4 import BeautifulSoup

In [9]:
url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
paragraphs = soup.find_all()
with open('webpage_content.txt', 'w', encoding='utf-8') as file:
    for paragraph in paragraphs:
        file.write(paragraph.text + '\n')

scrape for the urls:

In [34]:
main_url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy'
response = requests.get(main_url)

# Parse the HTML content and Find all <a> tags
soup = BeautifulSoup(response.text, 'html.parser')
links = soup.find_all('a')

# Extract the href attribute from each <a> tag and filter only relevant ones 
nested_urls = [link.get('href') for link in links if link.get('href')]
filtered_urls = [url for url in nested_urls if url.startswith(main_url + "/")]

nested_nested_urls = []
for secondary_url in filtered_urls:
    response = requests.get(secondary_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')

    temp = [link.get('href') for link in links if link.get('href')]
    nested_nested_urls += temp



In [62]:
# filter all the nested_nested_urls:
filtered_urls = [url for url in nested_nested_urls if url.startswith(main_url + "/")]

filtered_urls2 = [filtered_urls[0]]

for i in range(1, len(filtered_urls)):
    # Compare each element with the previous one
    if filtered_urls[i] != filtered_urls[i-1]:
        # Add it to the filtered list if it's different
        filtered_urls2.append(filtered_urls[i])


for all the content:

In [59]:
url = 'https://www.dpg-verhandlungen.de/year/2024/conference/berlin/part/dy/session/60/contribution/4'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
paragraphs = soup.find_all('p')

with open('webpage_content.txt', 'w', encoding='utf-8') as file:
    for paragraph in paragraphs:
        file.write(paragraph.text + '\n')

        


scaling the above to all

In [68]:
i=1

with open('all_text2.txt', 'w') as file:
    pass  # This clears the file

def append_to_file(content, filename='all_text2.txt'):
    with open(filename, 'a', encoding='utf-8') as file:
        file.write(content + '\n')

for url in filtered_urls2:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')

    append_to_file("abstract " + str(i))  
    for paragraph in paragraphs:
        append_to_file(paragraph.text)
    i+=1 

In [70]:
# cleaning:

unwanted_phrases = [
    "Auswahlstatus für diesen Beitrag: ",
    "Titel und präsentierender Autor",
    "Titel und Autorenblock",
    "Titel, Autorenblock und Kurzfassung",
    "gemäß den Sitzungseinstellungen"
]

def clean_file(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as infile, \
         open(output_filename, 'w', encoding='utf-8') as outfile:
        for line in infile:
            # Check if the line contains any of the unwanted phrases
            if not any(unwanted_phrase in line for unwanted_phrase in unwanted_phrases):
                outfile.write(line)

# Replace 'your_input_file.txt' with the path to your input file
# and 'cleaned_file.txt' with the desired output file path
clean_file('all_text2.txt', 'cleaned_file.txt')

Make pretier:

worked option


In [7]:
def format_abstracts(input_file, output_file):
    # Initialize variables
    current_title, current_authors, current_affiliations, current_abstract = "", "", "", ""
    processing_abstract = False

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    # Prepare to write to the output file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for line in lines:
            if line.strip().startswith('abstract'):
                if processing_abstract:  # If already processing an abstract, write it to the file
                    outfile.write(f"Title: {current_title}\n")
                    outfile.write(f"Authors: {current_authors}\n")
                    outfile.write(f"Affiliations: {current_affiliations}\n")
                    outfile.write("Abstract:\n")
                    outfile.write(current_abstract.strip() + "\n\n")
                # Reset for the next abstract
                current_abstract = ""
                processing_abstract = False
            elif not processing_abstract and line.strip():
                # Process the title, authors, and affiliations line
                parts = line.strip().split(' — ')
                if len(parts) >= 3:
                    current_title = parts[0]
                    current_authors = parts[1]
                    current_affiliations = ' — '.join(parts[2:])
                    processing_abstract = True
            elif processing_abstract:
                # Accumulate the abstract text
                current_abstract += line

        # Write the last abstract, if any
        if processing_abstract:
            outfile.write(f"Title: {current_title}\n")
            outfile.write(f"Authors: {current_authors}\n")
            outfile.write(f"Affiliations: {current_affiliations}\n")
            outfile.write("Abstract:\n")
            outfile.write(current_abstract.strip() + "\n")

# Specify your input and output files
input_file_path = 'cleaned_file.txt'
output_file_path = 'pretty.txt'

# Call the function
format_abstracts(input_file_path, output_file_path)

print('Formatting complete.')

Formatting complete.


making abstract in one line

In [19]:
def process_content_from_file(input_file_path, output_file_path):
    # Read the content from the input file
    with open(input_file_path, 'r', encoding='utf-8') as file:
        lines = file.read().strip().split('\n')

    output_lines = []
    is_abstract = False
    abstract_text = ""

    for line in lines:
        if line.startswith('Title:'):
            if abstract_text:  # If there's an abstract from the previous block, add it before starting a new title
                output_lines.append('Abstract:' + abstract_text.strip() + '\n')
                abstract_text = ""  # Reset abstract text for the next block
            is_abstract = False
            output_lines.append(line)
        elif line.startswith('Authors:') or line.startswith('Affiliations:'):
            is_abstract = False
            output_lines.append(line)
        elif line.startswith('Abstract:'):
            is_abstract = True
            if abstract_text:  # In case of consecutive abstract lines without a title in between
                output_lines.append('Abstract:' + abstract_text.strip() + '\n')
                abstract_text = ""
            abstract_text += line.replace('Abstract:', '').strip() + " "
        else:
            if is_abstract:
                abstract_text += line.strip() + " "
            else:
                output_lines.append(line)

    if abstract_text:  # Add the last abstract block if present
        output_lines.append('Abstract:' + abstract_text.strip())

    processed_content = '\n'.join(output_lines)

    # Write the processed content to the output file
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(processed_content)

# Specify the path to your input and output files
input_file_path = 'pretty.txt'  # Adjust this path if your file is in a different location
output_file_path = 'output.txt'

# Process the file and write the output
process_content_from_file(input_file_path, output_file_path)

print("Processing complete! Check output.txt for the results.")


Processing complete! Check output.txt for the results.


Tests:

In [11]:
a = [1,2]
b = [4, 3]
a += b
c = a
print(c)
print(a)

[1, 2, 4, 3]
[1, 2, 4, 3]


In [63]:
# listing all the urls that make sence  
with open('urls2.txt', 'w', encoding='utf-8') as file:
    for link in filtered_urls2:
        file.write(link + '\n')